## CNN(image classification)
###CIFAR-10 dataset
###transfer learning 중 ResNet50을 이용 (higher accuracy)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
import os
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import sys

print("tensorflow version: ", tf.__version__)
print("python version: ", sys.version)

tensorflow version:  2.8.0
python version:  3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]


In [ ]:
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

x_train=x_train.astype(np.float32)/255.0
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)
y_test=tf.keras.utils.to_categorical(y_test,10)

#x_train, _, y_train,_ = train_test_split(x_train, y_train, test_size=0.5, random_state=42)
x_val, _, y_val,_ = train_test_split(x_test, y_test, test_size=0.8, random_state=42)

input_shape = x_train.shape[1:]

In [ ]:
g_epoch = 80
g_batch = 128
np.random.seed(42)
tf.random.set_seed(42)

print("reduced train/val size:", len(x_train), len(x_val), "input shape:", input_shape)

reduced train/val size: 50000 2000 input shape: (32, 32, 3)


In [ ]:
from tensorflow.keras.layers import MaxPooling2D, Dropout, Conv2D

#C-P-drop-C-P-drop-C-C-P-fc-drop-fc
cnn=Sequential()
cnn.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3)))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(1000,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax'))

In [ ]:
cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 128)        0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)        

In [ ]:
hist=cnn.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

g_org_res=cnn.evaluate(x_test,y_test,verbose=0)
print("Baseline 정확률은",g_org_res[1]*100)

Epoch 1/80
391/391 [==============================] - 25s 37ms/step - loss: 2.1614 - accuracy: 0.1945 - val_loss: 1.9960 - val_accuracy: 0.3090
Epoch 2/80
391/391 [==============================] - 14s 35ms/step - loss: 1.8846 - accuracy: 0.3169 - val_loss: 1.9117 - val_accuracy: 0.3065
Epoch 3/80
391/391 [==============================] - 14s 36ms/step - loss: 1.7587 - accuracy: 0.3624 - val_loss: 1.7948 - val_accuracy: 0.3505
Epoch 4/80
391/391 [==============================] - 14s 35ms/step - loss: 1.6826 - accuracy: 0.3897 - val_loss: 1.7720 - val_accuracy: 0.3680
Epoch 5/80
391/391 [==============================] - 14s 35ms/step - loss: 1.6174 - accuracy: 0.4133 - val_loss: 1.6559 - val_accuracy: 0.4080
Epoch 6/80
391/391 [==============================] - 14s 35ms/step - loss: 1.5661 - accuracy: 0.4326 - val_loss: 1.7067 - val_accuracy: 0.4010
Epoch 7/80
391/391 [==============================] - 14s 36ms/step - loss: 1.5236 - accuracy: 0.4477 - val_loss: 1.5848 - val_accuracy:

In [ ]:
no_class = 10 #number of class

# for transfer learning only. using ResNet50
from tensorflow.keras.applications.resnet50 import ResNet50

# for transfer learning only
transfermodel = ResNet50(weights='imagenet',include_top=False,
                    input_shape=input_shape)
#base_model.trainable=False     # it's up to you

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
# your model architecture
model=Sequential()
model.add(transfermodel)    # for transfer learning only
model.add(Flatten())        # for transfer learning only
model.add(Dense(1000,activation='relu'))
model.add(Dense(no_class, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 1000)              2049000   
                                                                 
 dense_3 (Dense)             (None, 10)                10010     
                                                                 
Total params: 25,646,722
Trainable params: 25,593,602
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
hist=model.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

yours=model.evaluate(x_test,y_test,verbose=0)

Epoch 1/80
391/391 [==============================] - 70s 155ms/step - loss: 1.8932 - accuracy: 0.3740 - val_loss: 73.0072 - val_accuracy: 0.0905
Epoch 2/80
391/391 [==============================] - 57s 146ms/step - loss: 1.1268 - accuracy: 0.6094 - val_loss: 3.0514 - val_accuracy: 0.1840
Epoch 3/80
391/391 [==============================] - 57s 146ms/step - loss: 0.8414 - accuracy: 0.7068 - val_loss: 1.1205 - val_accuracy: 0.6225
Epoch 4/80
391/391 [==============================] - 57s 146ms/step - loss: 0.6542 - accuracy: 0.7751 - val_loss: 1.0823 - val_accuracy: 0.6480
Epoch 5/80
391/391 [==============================] - 57s 145ms/step - loss: 0.4989 - accuracy: 0.8327 - val_loss: 1.0750 - val_accuracy: 0.6635
Epoch 6/80
391/391 [==============================] - 58s 147ms/step - loss: 0.3817 - accuracy: 0.8765 - val_loss: 1.0842 - val_accuracy: 0.6645
Epoch 7/80
391/391 [==============================] - 57s 146ms/step - loss: 0.2848 - accuracy: 0.9121 - val_loss: 1.1055 - val_a

In [ ]:
print("Baseline vs yours: ",g_org_res[1]*100, yours[1]*100)

Baseline vs yours:  71.92000150680542 75.44999718666077
